In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: mps


In [3]:
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": device} if device == "mps" else "auto",
)

In [4]:
dataset_path = "/Users/priyal/Documents/honours/datasets/qa_dataset_cleaned.csv"
dataset = load_dataset("csv", data_files=dataset_path)

In [5]:
splits = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = splits["train"]
eval_dataset = splits["test"]

print(f"Training examples: {len(train_dataset)}")
print(f"Evaluation examples: {len(eval_dataset)}")

Training examples: 518
Evaluation examples: 58


In [6]:
def tokenize_function(examples):
    inputs = [
        f"Instruction: {inst}\nInput: {inp}"
        for inst, inp in zip(examples["instruction"], examples["input"])
    ]
    outputs = examples["output"]

    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=512,
    )

    labels = tokenizer(
        outputs,
        padding="max_length",
        truncation=True,
        max_length=128,  # T5 outputs are usually shorter
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_train = train_dataset.map(
    tokenize_function, batched=True, remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    tokenize_function, batched=True, remove_columns=eval_dataset.column_names
)

Map: 100%|██████████| 58/58 [00:00<00:00, 3145.54 examples/s]


In [8]:
peft_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q", "v"],
    bias="none",
    inference_mode=False,
)

In [9]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
trainable params: 4,718,592 || all params: 787,868,672 || trainable%: 0.5989


In [10]:
training_args = TrainingArguments(
    output_dir="./flan_t5_large_results",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    per_device_train_batch_size=2,  # Adjust if needed for memory
    gradient_accumulation_steps=8,  # Effective batch size of 16
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs_flan_t5_large",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="tensorboard",
    fp16=False,  # MPS doesn't support fp16
    dataloader_num_workers=4,
    remove_unused_columns=True,
    group_by_length=True,
)

/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
)

/var/folders/4y/9y5ld1xn3zn_rhkhdmmr63zr0000gn/T/ipykernel_71137/3279074610.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [12]:
print("Starting fine-tuning")
train_result = trainer.train()
trainer.save_model("./flan_t5_large_finetuned")
tokenizer.save_pretrained("./flan_t5_large_finetuned")

Starting fine-tuning


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
print(f"Training metrics: {train_result.metrics}")
final_eval = trainer.evaluate()
print(f"Final evaluation metrics: {final_eval}")

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    default_data_collator,
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(
    "./flan_t5_finetuned",
    torch_dtype=torch.float32,
    device_map={"": device} if device == "mps" else "auto",
)
tokenizer = AutoTokenizer.from_pretrained("./flan_t5_finetuned")

print("Model and tokenizer loaded successfully!")

/Users/priyal/Documents/honours/mistral_env2/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Model and tokenizer loaded successfully!


In [4]:
def generate_response(instruction, input_text="", max_new_tokens=500):
    prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput: "
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            num_beams=3,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
        )

    full_output = tokenizer.decode(output[0], skip_special_tokens=True)

    if prompt in full_output:
        return full_output[len(prompt) :].strip()
    elif "Output: " in full_output:
        return full_output.split("Output: ")[1].strip()
    return full_output


In [5]:
instruction = "Explain how Ayurvedic principles address the physiological and cognitive decline associated with aging."
input_text = """OBJECTIVES Various aspects of aging in Ayurveda literature has been explored and correlated it with modern biological understanding. CONTENT The concept of longevity is better understood upon the baseline understanding of Aging ie. Jara. Manda Jara or Ajara is synonymous with longevity. VriddhavasthaJirnavasthaJara old age is the period after 60 or 70 years of age. This stage of life becomes very crucial as degenerative changes occur in the body. Deterioration occurs to Dhatus, Indriya perception power, Bala strength, Virya potency, Utsaha enthusiasm, Paurusha masculinity, Parakrama valour, Grahana reception, Dharana retention, Smarana memory, Vachana speech. Brihatrayis and Laghutrayis were reviewed to analyze the various aspects of Ayu and Jara and correlated with research papers establishing the concept of aging in terms of evidence-based technological advancements. Databases like PubMed, Scopus were searched using keywords like Jara, Ayu Lakshana, Aging, Old age, Senescence, biomarkers, DNA, blood biochemistry, proteomics, genomics, microbiota, microbiome, androgen, estrogen, lipolysis, peripheral blood monocytes, collagen, cognitive function with the help of Boolean operators. SUMMARY Various aspects of Jara in Ayurveda have been analyzed by taking ample references from evidence-based technological advancements related to the physiology of aging. The indicators of biological aging based on recent researches like genomic irregularities, mitochondrial dysfunction, cellular senescence, altered protein expression, immunological markers, oxidative stress, pathological microbiota profile, telomere shortening in peripheral blood monocytes, lipolysis, and many other factors have been put forward for basic evidence in relation with Jara Lakshana. OUTLOOK Understanding the physiology of aging will give the insight to manage the elderly. So, the need of the hour is to develop strategies or to conduct more researches which prevent senile disorders and thus increase the health span for the future generation."""

response = generate_response(instruction, input_text)
print("\nGenerated response:")
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (538 > 512). Running this sequence through the model will result in indexing errors



Generated response:
Various aspects of aging in Ayurveda literature has been explored and correlated it with modern biological understanding. The concept of longevity is better understood upon the baseline understanding of Aging ie. Jara. Manda Jara or Ajara is synonymous with longevity.
